## Recommending movies using Collaborative Filtering 

In [9]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import hamming

In [10]:
import warnings
warnings.simplefilter(action='ignore',category=Warning)

In [11]:
df_ratings = pd.read_csv("recent_ratings.csv")

In [12]:
df_ratings.shape

(552, 4)

In [13]:
df_ratings.sample(5)

,userId,movieId,rating,timestamp
50,62,176371,4.0,1521488605
184,210,122906,4.5,1537632293
314,338,187541,1.0,1530148375
303,331,177765,4.0,1537157585
408,471,168252,4.5,1496669544


In [14]:
# Remove timestamp column  
df_ratings.drop(columns='timestamp', inplace=True)

In [15]:
df_ratings.sample(5)

,userId,movieId,rating
197,212,168250,2.5
28,50,175661,1.5
147,184,122916,4.5
98,111,170897,2.0
339,380,168254,4.0


In [16]:
# Get rating for each user and movie - userid is row label and movieid is column label 
ratings = df_ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating')

In [17]:
ratings.sample(5)

movieId,122896,122898,122906,122912,122916,122918,122926,143355,166534,167064,...,189381,189713,190183,190209,190215,191005,193581,193583,193585,193587
userId,,,,,,,,,,,,,,,,,,,,,
331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
252,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,NaN,NaN,4.5,NaN,4.5,5.0,4.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Hamming Distance
Measures how different two sequences are. It is % of disagreement. A value of 1 indicates sequences are very different, 0 indicates they are very similar.

In [18]:
# Find out hamming distance between two users
def hamming_distance(user1,user2):
    # Ratings of a user 
    try:
        user1_ratings = ratings.loc[user1,:]
        user2_ratings = ratings.loc[user2,:]
        distance = hamming(user1_ratings, user2_ratings)
    except:
        distance = np.NaN

    return distance    

In [19]:
def get_nearest_users(active_user, k = 10):
    all_users = pd.DataFrame(ratings.index)
    other_users = all_users[all_users.userId != active_user]
    other_users['distance'] = other_users['userId'].apply(lambda x: hamming_distance(active_user,x))
    
    # find out hamming distance and return users with low hamming distance from active user 
    return  other_users.sort_values(['distance'], ascending = True).userId[:k]

In [20]:
def get_recommended_movies(ratings,user,top=5):
    # Find out nearest neighbours
    nn_users = get_nearest_users(user,10)
    
    # Get ratings of other nearest neighbours(users)
    nn_ratings = ratings[ratings.index.isin(nn_users)]
    
    # Average ratings giving by nearest neighbours for other movies
    avg_ratings = nn_ratings.apply(np.nanmean).dropna()
    
    # Find out movies that user had already watched
    movies_watched = ratings.transpose()[user].dropna().index
    
    # remove movies that user already watched
    avg_ratings = avg_ratings[~ avg_ratings.index.isin(movies_watched)]
    
    # Findout top n movies based on avg ratings given by other nearest neighbours 
    top_movies = avg_ratings.sort_values(ascending=False).index[:top]
    
    movies = pd.read_csv('recent_movies.csv')
    # Return recommended movies 
    return movies[movies.movieId.isin(top_movies)].title

In [21]:
get_recommended_movies(ratings,305,10)

38                                         Gifted (2017)
51                                    Baby Driver (2017)
56                                Tickling Giants (2017)
64     Valerian and the City of a Thousand Planets (2...
73                                    Logan Lucky (2017)
97      Three Billboards Outside Ebbing, Missouri (2017)
122                                        Wonder (2017)
125                           The Disaster Artist (2017)
126                                      The Post (2017)
132    Too Funny to Fail: The Life and Death of The D...
Name: title, dtype: object

In [22]:
get_recommended_movies(ratings,433,5)

3               Avengers: Infinity War - Part I (2018)
12                        The Lego Batman Movie (2017)
15                       John Wick: Chapter Two (2017)
20                                 The Big Sick (2017)
97    Three Billboards Outside Ebbing, Missouri (2017)
Name: title, dtype: object